In [ ]:
from datetime import datetime
import os
import numpy as np
import netCDF4 as nc
from netCDF4 import Dataset
import pandas as pd
from netCDF4 import num2date
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import time
import sys
import cmaps
sys.path.append(r"/home/yfdong/data/work/wrf-hydro/coupled/Yangtze-Huai/code/Library")
from MeteoVarPlot import (add_quiverkey,
                          draw_2DVAR, draw_Bias, draw_VARtemporal, draw_Bias_subplot,draw_Prec_subplot ,
                          draw_2DuvVAR ,draw_uvBias_subplot ,draw_2DuvVAR_subplot
                        #   ,draw_Prec_temporal_subplot
                          )
from PreprocessVar import (
    get_MONTH_abbr, get_pentad, get_letter,extract_SubsetRegion_mask,
    preprocess_var, read_var,
    Read_VarsFromVarsDict, experiments_types,
    load_geodata, SubsetDomainConfigure ,create_domain_mask,get_letter, Meiyu_configs, 
    JJA_configs,Events_configs,get_letter,
    return_Event_Date,LST_Convert_to_UTC,get_indices_for_dates,save_CHM_PRE_daily,
    Events_configs_total ,getFileList,create_nc_file,read_cmorph_var,save_cmorph_hourly,calculate_metric
)

In [ ]:
# 读取数据
import xarray as xr
# 读取数据
BasePath = "/home/yfdong/data/work/wrf-hydro/coupled/Yangtze-Huai/"
CalibrationStatus = "def" 
VAR_NAME = 'RAINNC'
# 定义结果字典
years = [2011, 2018, 2020]

# 初始化结果字典
results_src_grid = {}
# 遍历每个年份
for year in years:
    # 为每个年份创建一个包含三个实验的字典
    results_src_grid[year] = {
        'WRF-S': {},
        'WRF-H': {},
        'CHM_PRE': {},
        'CMORPH': {}
    }
# 读取地形数据
Ctrl_lat, Ctrl_lon, LandMask, DEM = load_geodata(GeoPath ="/home/yfdong/data/work/wrf-hydro/coupled/Yangtze-Huai/output/Domain/def/geo_em.nc",IF_LandMask=True)
experiment = experiments_types[0]
# Events_config = JJA_configs[1]
for Events_config in JJA_configs:
    EventName = Events_config['EventName']
    print(Events_config)
    EventState='Total'
    Timedelta = 24 # Hours
    LAYER, SCALE_FACTOR = 0, 1
    # 读取事件的开始和结束时间
    ReadStartDate, ReadEndDate = return_Event_Date(EventState, Events_config)
    Year = ReadStartDate.year
    DateList = pd.date_range(start=ReadStartDate, end=ReadEndDate, freq=f'{Timedelta}h')

    # 读取WRF-S模拟数据
    DATA_PATH = os.path.join(BasePath, f'output/JJA/{EventName}/nc')
    results_src_grid[Year]['WRF-S'],_ = preprocess_var(DATA_PATH, CalibrationStatus, experiment['control_method'], 
                    VAR_NAME, DateList, ReadStartDate, ReadEndDate, LAYER, 
                    SCALE_FACTOR, LandMask, IF_LANDMASK=False)
    
    # 读取WRF-H模拟数据
    results_src_grid[Year]['WRF-H'],_ = preprocess_var(DATA_PATH, CalibrationStatus, experiment['experiment_method'], 
                    VAR_NAME, DateList, ReadStartDate, ReadEndDate, LAYER, 
                    SCALE_FACTOR, LandMask, IF_LANDMASK=False)   
     
    # 读取CHM_PRE数据
    dstFilePath = os.path.join(BasePath, f'obs/CHM_PRE/JJA_{Year}_CHM_PRE_1-Daily.nc')
    FilePath = dstFilePath   
    results_src_grid[Year]['CHM_PRE'], CHM_lat, CHM_lon = read_cmorph_var(dstFilePath, ReadStartDate, ReadEndDate, 'Prec' ,SCALE_FACTOR )

    # 读取CMORPH数据
    dstFilePath = os.path.join(BasePath, f'obs/CMORPH/JJA_{Year}_cmorph_1-Hourly.nc')
    Var, CMORPH_lat, CMORPH_lon = read_cmorph_var(dstFilePath, ReadStartDate, ReadEndDate, 'Prec' ,SCALE_FACTOR )
    # -------------------- CMORPH 转化为daily数据 ------------------
    CMORPH_DateList = pd.date_range(start=f'{year}-06-01', end=f'{year}-08-30', freq='1h')
    # 创建 xarray Dataset
    ds = xr.Dataset(
        {
            "Prec": (["time", "lat", "lon"], Var)
        },
        coords={
            "time": CMORPH_DateList,
            "lat": CMORPH_lat,
            "lon": CMORPH_lon
        }
    )
    # 使用 resample 方法计算日均值
    daily_ds = ds.resample(time="1D").mean()
    # 提取日均值数据
    results_src_grid[Year]['CMORPH'] = daily_ds['Prec'].values
    
    
    

In [ ]:
# 统一分辨率
'''
To assess the performance of precipitation simulations, 
both the CHM_PRE and WRF-S model outputs were spatially interpolated onto the CMORPH grid using a bilinear interpolation technique
Observation data: CHM_PRE and CMORPH, while the WRF-S is model output.
'''
from scipy.interpolate import griddata, RegularGridInterpolator
# 初始化结果字典
results_CMORPH_grid = {}
# 遍历每个年份
for year in years:
    # 为每个年份创建一个包含三个实验的字典
    results_CMORPH_grid[year] = {
        'WRF-S': {},
        'CHM_PRE': {},
        'CMORPH': {}
    }


# print(test.shape,src_lat, src_lon)
# --------------------- 提取子区域 --------------------
def create_domain_mask(lat, lon, domain):
   domain_lat_min, domain_lat_max, domain_lon_min, domain_lon_max = domain
   mask = (lat >= domain_lat_min) & (lat <= domain_lat_max) & (lon >= domain_lon_min) & (lon <= domain_lon_max)
   masked_array = xr.DataArray(np.where(mask, 1, np.nan), dims=[ 'south_north', 'west_east'])
   return masked_array
def extract_SubsetRegion_mask(var ,domain_mask):
    row_indices, col_indices = np.where(domain_mask == 1)  # 提取 ==1 的区域
    row_start, row_end = row_indices.min(), row_indices.max()
    col_start, col_end = col_indices.min(), col_indices.max()
    # 步骤2：切片获取子区域数据（保持二维结构）
    sub_var = var[row_start:row_end+1, col_start:col_end+1]
    return sub_var
DomainRegion = [23, 40, 106, 125] # 计算纬向平均的经纬度范围
# DomainRegion = [24, 35, 105, 126] 
dst_lon_grid , dst_lat_grid= np.meshgrid(CMORPH_lon, CMORPH_lat)
DomainMask = create_domain_mask(dst_lat_grid, dst_lon_grid, DomainRegion)
# 提取子区域
DomainMask = np.expand_dims(DomainMask,0) #广播
print(dst_lat_grid.shape, dst_lon_grid.shape)
# -------------------- 统一分辨率 ----------------------------
def interpolate_to_cmorph(src_var, src_lat, src_lon, CMORPH_lat, CMORPH_lon):
    
    dst_lat, dst_lon = CMORPH_lat, CMORPH_lon
    dst_lon_grid, dst_lat_grid = np.meshgrid(dst_lon, dst_lat)
    dst_points = np.array([dst_lat_grid.flatten(), dst_lon_grid.flatten()]).T
    
    # 强制类型转换
    dst_points = dst_points.astype(np.float32)
    REGRIDmethod = "linear"
    REGRIDmethod = "nearest"
    if src_var.shape[0]== 1 or src_var.ndim==2:
        dst_var = np.zeros((int(len(dst_lat)), int(len(dst_lon))))
        print('SRC',src_var.shape,src_lat.shape, src_lon.shape)
        print('dst',dst_var.shape, dst_lat.shape, dst_lon.shape)
        interpolator = RegularGridInterpolator((src_lat, src_lon), src_var, method=REGRIDmethod,bounds_error=False, fill_value=None)
        dst_var[:] = interpolator(dst_points).reshape((len(dst_lat), len(dst_lon)))
    elif src_var.shape[0] > 1 or src_var.ndim>=2:
        dst_var = np.zeros((int(src_var.shape[0]), int(len(dst_lat)), int(len(dst_lon))))
        for DateIndex in range(len(DateList)):  
            interpolator = RegularGridInterpolator((src_lat, src_lon), src_var[DateIndex,:], method=REGRIDmethod,bounds_error=False, fill_value=None)
            dst_var[DateIndex,:] = interpolator(dst_points).reshape((len(dst_lat), len(dst_lon)))
    else:
        print('Error: src_var.shape[0] is not 1 or greater than 1')
    return dst_var    
years = [2011,2018,2020]
Vars = ['WRF-S', 'WRF-H','CHM_PRE']
for yearIndex, year in enumerate(years):
    for varIndex, var in enumerate(Vars):
        print(year, var)
        # 读取源数据
        src_var = results_src_grid[year][var]
        if var=='WRF-S' or var=='WRF-H':
            src_lon, src_lat = Ctrl_lon[1,:], Ctrl_lat[:,1]
        elif var=='CHM_PRE':
            src_lon, src_lat = CHM_lon, CHM_lat
        # 进行插值计算
        interpolate_data= interpolate_to_cmorph(src_var, src_lat, src_lon, CMORPH_lat, CMORPH_lon)
        interpolate_data[interpolate_data>1000]=np.nan
        interpolate_data[interpolate_data<0]=np.nan
        results_CMORPH_grid[year][var] = interpolate_data *DomainMask
    results_CMORPH_grid[year]['CMORPH'] = results_src_grid[year]['CMORPH'] *DomainMask
# 插值地形
dstDEM = interpolate_to_cmorph(DEM, Ctrl_lat[:,1], Ctrl_lon[1,:], CMORPH_lat, CMORPH_lon)
dstLandMask = interpolate_to_cmorph(LandMask, Ctrl_lat[:,1], Ctrl_lon[1,:], CMORPH_lat, CMORPH_lon)
        # # 计算均方根误差（RMSE）
        # RMSE = np.sqrt(np.mean((results_CMORPH_grid[year]['WRF-S'] - results_CMORPH_grid[year]['CMORPH'])**2))
        # # 计算均方根误差（RMSE）
        # RMSE = np.sqrt(np.mean((results_CMORPH_grid[year]['WRF-S] - results_CMORPH_grid[year]['CHM_PRE'])**2))
        # # 计算相关系数
        # CORR = np.corrcoef(results_CMORPH_grid[year]['WRF-S'].flatten(), results_CMORPH_grid[year]['CMORPH'].flatten())[0, 1]
        # # 计算相关系数
        # CORR = np.corrcoef(results_CMORPH_grid[year]['CHM_PRE'].flatten(), results_CMORPH_grid[year]['CMORPH'].flatten())[0, 1]


In [ ]:
# 可视化
import cartopy.feature as cfeature
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cmaps
CMAP = cmaps.WhiteBlueGreenYellowRed
Vars = ['WRF-S', 'WRF-H', 'CMORPH']
years = [2011,2018,2020]
# 定义公共参数
Plt_CommonParams = {
    'width': 3.2,
    'Height': 2.3,
    'dpi': 250,
    'title_labelsize': 6,
    'xy_labelsize': 7,
    'cbar_Legend_size': 6,
    'Grid': True,
    'Save': False,
    'Shp': False,
    'River': False,
    'Basin': False,
    'Topo': True,
    'Cbar': False
}
i = 0
# 修改部分1：在开始绘图前设置全局rcParam参数
plt.rcParams['axes.edgecolor'] = 'grey'  # 边框颜色
plt.rcParams['axes.linewidth'] = 0.5     # 边框线宽
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12/1.4/1.1, 7/1.1), dpi=200 ,sharex=True ,subplot_kw={'projection': ccrs.PlateCarree()})
# axs = axes.flatten()
DomainRegion = [25,38,107.5,124] # 绘图范围
plotMIN ,plotMAX1 = 0, 16
subsample = 1
# 用于存储所有图像对象的列表
images = []
for yearIndex, year in enumerate(years):
    for varIndex, var in enumerate(Vars):
        # print(yearIndex, year, varIndex, var)
        Index = yearIndex
        if var=='WRF-S':
            Var = np.nanmean(results_CMORPH_grid[year][var], axis =0)
            lon, lat = Ctrl_lon, Ctrl_lat
        elif var=='WRF-H':
            # print('CHM_PRE',results_CMORPH_grid[year][var].shape)
            Var = np.nanmean(results_CMORPH_grid[year][var], axis =0)
            lon, lat = CHM_lon, CHM_lat
        elif var=='CMORPH':
            # print('CMORPH',results_src_grid[year][var].shape)
            Var = np.nanmean(results_src_grid[year][var], axis =0)*24 # (mm/hour)->(mm/day)
            lon, lat = CMORPH_lon, CMORPH_lat
        else:
            raise ValueError(f"Unknown variable: {var}")
        lon, lat = CMORPH_lon, CMORPH_lat

        ax=axes[varIndex, yearIndex]
        # print(dstDEM.shape, Var.shape, lon.shape, lat.shape)
        # 调用绘图函数并存储图像对象
        im = draw_Prec_subplot(
            DomainRegion, plotMIN, plotMAX1, lon, lat,
            Var, None, cbarTitle=None,
            CMAP=CMAP, save_name=False,
            **Plt_CommonParams, DEM=dstDEM, ax=ax, IF_Rectangle=False
        )
        images.append(im)
        
        # 设置地图特征
        # ax.set_extent(DomainRegion, crs=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=0.5, zorder=2)  # 绘制黑色海岸线
        ax.add_feature(cfeature.OCEAN, facecolor='white', zorder=2)  # 海洋填充为白色
        if yearIndex==0:
            ax.set_ylabel(var, fontsize=Plt_CommonParams['xy_labelsize']+1, rotation=0, labelpad=30) #,fontweight='bold'
        if varIndex==0:
            ax.set_title(year, fontsize=Plt_CommonParams['xy_labelsize']+1 ) #,fontweight='bold'
            
        # ax.tick_params(axis='both', colors='gray', labelsize=8)  # 刻度线颜色和标签大小
        # ax.gridlines(color='gray', linestyle='--', linewidth=0.5, alpha=0.5)  # 可选：网格线
        if varIndex <2:
            # axe.set_xticks([])
            # axe.set_xticklabels([])  # 同步清除标签
            ax.tick_params(axis='x', which='both', bottom=False, labelbottom=False)
            # print("当前x轴刻度：", axe.get_xticks())   # 应该输出空数组
            if yearIndex != 0:
                ax.set_yticks([])
        elif varIndex ==2:
            if yearIndex != 0:
                ax.set_yticks([])        
# 创建总的colorbar
cax = fig.add_axes([0.2, 0.05, 0.5, 0.015])
# ticks = np.arange(0, 21, 5)
cbar = fig.colorbar(images[0], cax=cax, orientation='horizontal',
                    extend=None,  # 可选：显示超出范围的颜色
                    ticks=np.arange(plotMIN, plotMAX1+1, 5),# 设置刻度
                    cmap = CMAP,
                    )
# 设置颜色条的刻度标签字体大小
cbar.ax.tick_params(labelsize=Plt_CommonParams['xy_labelsize'])
# fig.colorbar(images[0], cax=cax, label='Your Colorbar Label',
#              extend='both',  # 可选：显示超出范围的颜色
#             #  ticks=np.arange(plotMIN, plotMAX1+1, 5),# 设置刻度
#             cmap = CMAP)  
# cbar.set_ticks(ticks)
# 设置colorbar标题（如果需要）
cbar.ax.set_xlabel('(mm/day)', fontsize=Plt_CommonParams['xy_labelsize']+1 )
# 调整子图间距
plt.subplots_adjust( wspace=0.05,hspace=0.05) #bottom=0.1, top=0.96,
plt.show()

In [ ]:
# 哈默图，雨带的纬向传播
import xarray as xr
from matplotlib import ticker
import matplotlib.dates as mdates
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.patches import Rectangle

# --------------------- 提取子区域 --------------------
def create_domain_mask(lat, lon, domain):
   domain_lat_min, domain_lat_max, domain_lon_min, domain_lon_max = domain
   mask = (lat >= domain_lat_min) & (lat <= domain_lat_max) & (lon >= domain_lon_min) & (lon <= domain_lon_max)
   masked_array = xr.DataArray(np.where(mask, 1, np.nan), dims=[ 'south_north', 'west_east'])
   return masked_array
def extract_SubsetRegion_mask(var ,domain_mask):
    row_indices, col_indices = np.where(domain_mask == 1)  # 提取 ==1 的区域
    row_start, row_end = row_indices.min(), row_indices.max()
    col_start, col_end = col_indices.min(), col_indices.max()
    # 步骤2：切片获取子区域数据（保持二维结构）
    sub_var = var[row_start:row_end+1, col_start:col_end+1]
    return sub_var
DomainRegion = [23, 40, 110, 122] # 计算纬向平均的经纬度范围
dst_lon_grid , dst_lat_grid= np.meshgrid(CMORPH_lon, CMORPH_lat)
DomainMask = create_domain_mask(dst_lat_grid, dst_lon_grid, DomainRegion)
# 提取子区域
DomainMask = np.expand_dims(DomainMask,0) #广播
print(dst_lat_grid.shape, dst_lon_grid.shape)
def add_cbar(Cbar, ax, fig, contour, cbar_Legend_size, cbarTitle ,numticks=5):
    if Cbar==True:
        
        # # ----------------------------绘制colorbar----------------------------
        # 指定colorbar的位置和大小
        cax = fig.add_axes([0.35, 0.2, 0.3, 0.01])
        # 第一个参数：子图左下角的x坐标，表示子图距离整个图形左边界的距离占整个图形宽度的比例。
        # 第二个参数：子图左下角的y坐标，表示子图距福整个图形底边界的距离占整个图形高度的比例。
        # 第三个参数：子图的宽度，表示子图宽度占整个图形宽度的比例。
        # 第四个参数：子图的高度，表示子图高度占整个图形高度的比例。
        cbar = fig.colorbar(contour, shrink=0.6,fraction=0.04, pad=0, orientation='horizontal', ax=ax,cax=cax)#, ticks=levels)
        # 设置colorbar的标题
        bbox = cbar.ax.get_position()
        title_pos = (bbox.x0 + bbox.width / 1+0.04, bbox.y0 + bbox.height / 2)
        ax.text(title_pos[0], title_pos[1], cbarTitle, fontsize=cbar_Legend_size,
                rotation=0, va='center', ha='center', transform=fig.transFigure)
    elif Cbar == "subcbar":
        # fraction控制颜色条在垂直于其方向上的大小。例如，对于水平方向的颜色条，fraction 控制的是颜色条的高度；对于垂直方向的颜色条，fraction 控制的是颜色条的宽度
        cbar = fig.colorbar(contour,  ax=ax, orientation='horizontal', shrink=1.5 ,fraction=0.025, pad=0.12, aspect=35)    
    cbar.locator = ticker.LinearLocator(numticks=numticks)
    # 设置刻度线的长度和方向
    cbar.ax.tick_params(which='both', length=1.2) #, direction='in'
    cbar.ax.tick_params(labelsize=cbar_Legend_size-1.5)
    
    # 自定义格式化函数
    def format_tick(tick_value, pos):
        if isinstance(tick_value, float) and not tick_value.is_integer():
            return f"{tick_value:.2f}"
        else:
            return str(int(tick_value)) if isinstance(tick_value, float) and tick_value.is_integer() else str(tick_value)
    # 自定义格式化函数
    def format_tick(tick_value, pos):
        if isinstance(tick_value, (float)):
            if tick_value > 100:
                return str(int(tick_value))
            else:
                return f"{tick_value:.2f}"

        else:
            return str(tick_value)
    
    # 使用自定义格式化函数
    cbar.formatter = ticker.FuncFormatter(format_tick)
    cbar.update_ticks()
    pass

def draw_Prec_temporal_subplot(
                Domain, plotMAX ,CMAP ,
                LAT, VAR , DateVAR, title, cbarTitle, save_name=True,
                Height = 2.3, width=3.2,
                dpi= 350,
                title_labelsize = 9,xy_labelsize = 8, cbar_Legend_size = 6, date_interval = "'weekly",
                Grid = True, Save=False, 
                ax=None, Cbar=False):  

    import matplotlib.dates as mdates
    # --------------------设置图片参数--------------------------
    print(save_name)
    # 检查ax是否为None
    if ax is None:
        # 创建新的Figure和Axes对象
        fig = plt.figure(figsize=(width, Height), dpi=dpi)
        ax = fig.subplots(1, 1, subplot_kw={'projection': ccrs.PlateCarree()})
    else: #绘制子图
        # 使用传入的ax
        fig = ax.figure
    # 设置绘图区背景颜色
    ax.set_facecolor('#F5F5F5')
    # 设置字体大小
    #print(LAT)
    levels = np.linspace(1, plotMAX, 31)
    
    # 转置数组
    VAR_transposed = VAR.T
    norm = BoundaryNorm(levels, ncolors=CMAP.N)
    # 将日期转换为数字格式
    DateList_num = mdates.date2num(DateList)
    contour = ax.contourf(DateList_num, lat, (Var.T), cmap=CMAP, levels=levels, extend='max', norm=norm)
    # ----------------- 绘制虚线框 -----------------
    # 添加红色虚线框
    # 获取 MeiyuStartDate 和 MeiyuEndDate 在 DateList_num 中的位置
    start_date_num = mdates.date2num(MeiyuStartDate)
    end_date_num = mdates.date2num(MeiyuEndDate)
    # # 计算纬度范围
    # lat_range = EndLat - StartLat
    # rect = Rectangle((start_date_num, StartLat), end_date_num - start_date_num, lat_range,
    #                 edgecolor='red', facecolor='none', linestyle='--', linewidth=2)
    # ax.add_patch(rect)
    import matplotlib.dates as mdates
    # 设置日期格式
    if date_interval == 'daily':
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
    elif date_interval == 'weekly':
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
    elif date_interval == 'monthly':
        ax.xaxis.set_major_locator(mdates.MonthLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m'))
    elif date_interval == 'yearly':
        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    # -----------绘制colorbar-----------
    if Cbar == True:
        add_cbar(Cbar, ax, fig, contour, cbar_Legend_size, cbarTitle,numticks=6)
    # 绘制标题
    ax.set_title(title , fontsize= title_labelsize, loc='left', x=-0.1)
    # 设置刻度的大小和标签的大小
    # 设置x轴为日期格式
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
    # ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=14))
    # 调整刻度线参数：
    ax.minorticks_on()
    # 显示副刻度线
    ax.tick_params(axis="both", which="major", direction="in", width=0.9, length=5, color='#363636')
    ax.tick_params(axis="both", which="minor", direction="in", width=0.75, length=3, color='#363636')
    # 设置主、副刻度线参数
    # grid 设置网格线性
    ax.grid(True, which="major", linestyle="--", color="gray", linewidth=0.75, alpha = 0.3)
    ax.grid(True, which="minor", linestyle=":", color="gray", linewidth=0.75, alpha = 0.3)
    #  设置绘图范围
    LATmin, Latmax, LONmin, LONmax = Domain
    ax.set_ylim(LATmin, Latmax)
    ax.set_ylim(24, 38) 


    return contour, levels



plotMIN ,plotMAX1 = 0, 30
images = []
years = [2011, 2018, 2020]
# years = [2020]
Vars = ['WRF-S', 'CHM_PRE', 'CMORPH']
Vars = ['WRF-S', 'WRF-H', 'CMORPH']
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(5.*3, 6), dpi=200 ,sharex=True ,sharey=True)
for yearIndex, year in enumerate(years):
    # 读取梅雨日期
    for Meiyu_config in Meiyu_configs:
        MeiyuStartDate, MeiyuEndDate = pd.to_datetime(Meiyu_configs[f'Meiyu-{year}']['StartDate']), pd.to_datetime(Meiyu_configs[f'Meiyu-{year}']['EndDate'])

    
    for varIndex, var in enumerate(Vars):
        # print(yearIndex, year, varIndex, var)
        Index = yearIndex
        start = time.time()
        if var=='WRF-S' or var=='WRF-H':
            Var = np.nanmean(results_CMORPH_grid[year][var]*DomainMask, axis =2)
        elif var=='CHM_PRE':
            Var = np.nanmean(results_CMORPH_grid[year][var]*DomainMask, axis =2)
        elif var=='CMORPH':
            Var = np.nanmean(results_src_grid[year][var]*DomainMask, axis =2)*24  
        else:
            raise ValueError(f"Unknown variable: {var}")
        lon, lat = CMORPH_lon, CMORPH_lat
        ax=axes[varIndex, yearIndex]
        # 调用绘图函数并存储图像对象
        contour, levels =draw_Prec_temporal_subplot(
                        DomainRegion, plotMAX1 ,CMAP ,
                        lat, Var , DateList, title=None, cbarTitle=None, save_name=True,
                        Height = 2.3, width=3.2,
                        dpi= 350,
                        title_labelsize = 9,xy_labelsize = 8, cbar_Legend_size = 6, date_interval = "'weekly",
                        Grid = True, Save=False, 
                        ax=ax, Cbar=False)
        if yearIndex==0:
            ax.set_ylabel(var, fontsize=Plt_CommonParams['xy_labelsize']+3, rotation=0, labelpad=30,fontweight='bold')
        if varIndex==0:
            ax.set_title(year, fontsize=Plt_CommonParams['xy_labelsize']+3 ,fontweight='bold')
        end = time.time()

    # 创建总的colorbar
    cax = fig.add_axes([0.2, 0.01, 0.6, 0.02])
    # ticks = np.arange(0, 21, 5)
    cbar = fig.colorbar(contour, cax=cax, orientation='horizontal',
                        extend=None,  # 可选：显示超出范围的颜色
                        ticks=np.arange(plotMIN, plotMAX1+1, 5),# 设置刻度
                        cmap = CMAP,
                        boundaries=levels, drawedges=True
                        )
    # 设置颜色条的刻度标签字体大小
    cbar.ax.tick_params(labelsize=Plt_CommonParams['xy_labelsize']+2)
    # 设置colorbar标题（如果需要）
    cbar.ax.set_xlabel('(mm/day)')
    cbar.dividers.set_color('gray')
    cbar.dividers.set_linewidth(0.3)  # 可选：设置分界线的宽度
    # 隐藏 colorbar 的刻度线，但保留数字标签
    cbar.ax.tick_params(size=0)  # 隐藏刻度线
    cbar.ax.yaxis.set_ticks_position('right')  # 将刻度标签移到右侧（可选）

    # # 调整子图间距
plt.subplots_adjust( wspace=0.05,hspace=0.1) #bottom=0.1, top=0.96,
# plt.show()